<center> <h1 style="background-color:firebrick; color:white" >Data Analysis of Boston Crimes</h1> 

<center><img
src="https://www.metro.us/wp-content/uploads/2020/03/crime_tape_875.jpg" style="width:90%;height:40%;">
</center>
    
<br>    
    
<div class="list-group" id="list-tab" role="tablist">
  <h3 class="list-group-item list-group-item-action active" style="background-color:firebrick; color:white" data-toggle="list"  role="tab" aria-controls="home">Content</h3>  
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#libraries" role="tab" aria-controls="profile" style="color:firebrick">Description<span class="badge badge-primary badge-pill" style="background-color:orangered; color:white">1</span></a>
  <a id="section2" class="list-group-item list-group-item-action" data-toggle="list" href="#functions" role="tab" aria-controls="messages" style="color:firebrick">Packages<span class="badge badge-primary badge-pill" style="background-color:orangered; color:white">2</span></a>
 <a id="section2" class="list-group-item list-group-item-action" data-toggle="list" href="#load" role="tab" aria-controls="messages" style="color:firebrick">EDA and Data Visualization<span class="badge badge-primary badge-pill" style="background-color:orangered; color:white">3</span></a>
 <a class="list-group-item list-group-item-action"  data-toggle="list" href="#missing" role="tab" aria-controls="settings" style="color:firebrick">General Analysis<span class="badge badge-primary badge-pill" style="background-color:orangered; color:white">4</span></a>
 <a class="list-group-item list-group-item-action"  data-toggle="list" href="#datatypes" role="tab" aria-controls="settings" style="color:firebrick">Modelling<span class="badge badge-primary badge-pill" style="background-color:orangered; color:white">5</span></a>
  

<a id='libraries'></a>
<h1 style="color:firebrick" >1. Description</h1>

Crime incident reports are provided by Boston Police Department (BPD) to document the initial details surrounding an incident to which BPD officers respond. This is a dataset containing records from the new crime incident report system, which includes a reduced set of fields focused on capturing the type of incident as well as when and where it occurred.
  
In this data, we will try to make predictions about future crimes by examining the criminal records in the Boston area.It would make sense to ask the following questions:

What types of crimes are most common?
Where are different types of crimes most likely to occur?
Does the frequency of crimes change over the day? Week? Year?

You will find these questions and general analysis at the bottom of the notebook.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<a id='libraries'></a>
<h1 style="color:firebrick" >2. Packages</h1>

In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
from pandas.api.types import CategoricalDtype
import plotly.graph_objects as go
import plotly.express as px
import datetime
from plotly.subplots import make_subplots
import folium
from folium.plugins import HeatMap
import pandas_profiling

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)
pd.set_option("display.float_format",lambda x: "%.5f" % x)
pd.set_option("display.max_columns",None)


from sklearn import preprocessing
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/kaggle/input/crimes-in-boston/crime.csv', encoding='latin-1')
df.head()

<a id='libraries'></a>
<h1 style="color:firebrick" >3. EDA and Data Visualization</h1>

In [ ]:
def check_df(dataframe):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(3))
    print("##################### Tail #####################")
    print(dataframe.tail(3))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

<font color = "#B32636">
Here we draw the following conclusion. There is no information for the first 5 months of 2015. There is no information for the last two months for 2018 either. Missing values for UCR_PART are due to this.
I will use visualization and proportioning to achieve a meaningful result in Na values.

In [ ]:
# Na values
import missingno as msno
msno.matrix(df)
plt.show()

In [ ]:
total_missing = df.isnull().sum().sort_values(ascending=False)
percent_missing = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing = pd.concat([total_missing, percent_missing], axis=1, keys=['total_missing', 'percent_missing'])
missing

<font color = "#B32636">
Here, the missing variable in Shooting has been deleted because it is too much.Since I will continue with the INDICENT NUMBER, delete GROUP_CODE, changed the type of DATE variable to datetime.I added a day of week variable.

In [ ]:
df = df.drop(['SHOOTING','OFFENSE_CODE'], axis=1)

df['OCCURRED_ON_DATE'] = pd.to_datetime(df['OCCURRED_ON_DATE'])

df["DAY_OF_WEEK"] = pd.Categorical(df["DAY_OF_WEEK"], 
              categories=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],
              ordered=True)

<font color = "#B32636">
I have derived new variables to make the week, month, year variables more understandable.

In [ ]:
rename = {'OFFENSE_CODE_GROUP':'Group',
          'OFFENSE_DESCRIPTION':'Description',
          'DISTRICT':'District',
          'STREET':'Street',        
          'OCCURRED_ON_DATE':'Date',
          'YEAR':'Year',
          'MONTH':'Month',
          'DAY_OF_WEEK':'Day',
          'HOUR':'Hour'}

df.rename(index=str, columns=rename, inplace=True)

# new feature
def create_features(df):
    df['dayofweek'] = df['Date'].dt.dayofweek
    df['quarter'] = df['Date'].dt.quarter
    df['dayofyear'] = df['Date'].dt.dayofyear
    df['dayofmonth'] = df['Date'].dt.day
    df['weekofyear'] = df['Date'].dt.weekofyear
    
    X = df[['dayofweek','quarter','dayofyear',
            'dayofmonth','weekofyear']]
    return X
create_features(df).head()

# CategoricalDytpe
df.quarter    = df.quarter.astype(CategoricalDtype())
df.dayofweek    = df.dayofweek.astype(CategoricalDtype())
df.dayofyear    = df.dayofyear.astype(CategoricalDtype())
df.dayofmonth    = df.dayofmonth.astype(CategoricalDtype())


df.head()

In [ ]:
df.apply(pd.Series.nunique)

<font color = "#B32636">
When the variable numbers of the data are examined carefully, it is seen that there are repetitive values.
Let's delete these duplicates.

In [ ]:
df.drop_duplicates(subset="INCIDENT_NUMBER", inplace=True)

In [ ]:
# descriptive analysis
df.describe().T

<font color = "#B32636">
The missing values in the year variables I stated earlier are visualized.(There is no information for the first 5 months of 2015. There is no information for the last two months for 2018 either.)

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
with sns.color_palette("RdBu",4):
  montyearAggregated = pd.DataFrame(df.groupby(["Month","Year"])["INCIDENT_NUMBER"].count()).reset_index()
  a=sns.barplot(data=montyearAggregated,x="Month", y="INCIDENT_NUMBER",hue = 'Year')
  a.set_title("Crime",fontsize=20)
  plt.legend(loc='upper right')
  plt.show()

In [ ]:
def custom_pallete(custom_colors):
    customPalette = sns.set_palette(sns.color_palette(custom_colors))
    sns.palplot(sns.color_palette(custom_colors),size=0.8)
    plt.tick_params(axis='both', labelsize=0, length = 0)

In [ ]:
red = ["#B32636","#4f000b","#720026","#ce4257","#ff7f51","#ff9b54"]
bo = ["#6930c3","#5e60ce","#0096c7","#48cae4","#ade8f4","#ff7f51","#ff9b54","#ffbf69"]
pink = ["#aa4465","#dd2d4a","#f26a8d","#f49cbb","#ffcbf2","#e2afff","#ff86c8","#ffa3a5","#ffbf81","#e9b827","#f9e576"]
custom_pallete(bo)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
fig.suptitle('ICU Type and Stay Type Distribution', size = 50,fontsize=30, color = red[0])
axs = [ax1, ax2]

explode = (0.1, 0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,0.05,0.05)
labels = ['JUNE', 'JULY', 'AUG','MAY', 'JAN', 'FEB','MARCH','APRIL','SEP','OCT','NOV','DEC']
sizes = df['Month'].value_counts()
ax1.pie(sizes, explode=explode, colors=pink[1:], startangle=60, labels=labels,autopct='%15.0f%%', pctdistance=0.6)
ax1.add_artist(plt.Circle((1,5),0.4,fc='white'))

explode = (0.1, 0.1, 0.05,0.05)
labels = ['2016','2017','2018','2015']
sizes = df['Year'].value_counts()
ax2.pie(sizes, explode=explode, colors=bo[2:], startangle=60, labels=labels,autopct='%15.01f%%', pctdistance=0.8)
ax2.add_artist(plt.Circle((0,0),0.4,fc='white'))

plt.show()


<font color = "#B32636">
With the last two images;
There seems to be an increase in cases from 2015 to 2016. This is normal considering the missing values in 2015. The year 2017 was the year with the highest crime rate and the crime rate is decreasing in 2018. Please consider the missing data for the last three months in 2018.
Although the months with the highest crime rates vary according to years, an increase was observed in the summer months with a general rate.

<font color = "#B32636">
The distribution of the crimes committed by district is shown.


In [ ]:
rcParams["figure.figsize"] = 18,7
sns.set(font_scale=1.75)
order = df['Group'].value_counts().head(5).index
sns.countplot(data = df, x='Group',hue='District', order = order,  saturation=2,linewidth=1)


<font color = "#B32636">
Here the highest crimes differ for each region.
The top 6 regions have the highest crime rates.
As for crimes, Motor Vehicle Accident Response, Larceny, Medical Assistance Group and Investigate Person are the most common ones.

<p><img style="float: left;margin:50px 50px 50px 50px; max-width:400px" src="https://www.verywellmind.com/thmb/XsiLFhGmbpHMhdcjPiCg_Ug6Ung=/2000x1334/filters:fill(ABEAC3,1)/GettyImages-94474503-56ad2bc53df78cf772b68346.jpg"></p>


<p><img style="float: right;margin:50px 50px 50px 50px; max-width:400px" src="https://ngm.com.au/wp-content/uploads/2019/05/grand-larceny-rsz.jpg"></p>


In [ ]:
grouped = df.groupby(['Month','District']).count()
sns.boxplot(x ="Month", y = "Group", data = grouped.reset_index(), palette="ch:.102");

<font color = "#B32636">
Looking at the month variable in the regional distribution, the crime rate is 6., 7. and 8. months.

In [ ]:
def hist(data,path,color,title,xaxis,yaxis):
    fig = px.histogram(data, x=path,color=color,nbins=70)
    fig.update_layout(
        title_text=title,
        xaxis_title_text=xaxis, 
        yaxis_title_text=yaxis, 
        bargap=0.5, 
        bargroupgap=0.5,
        
        
    )
    fig.show()
    
hist(df,"Group","Group",'Crimes in Boston','Crime','Count')

In [ ]:
from wordcloud import WordCloud
text = []
for i in df.Group:
    text.append(i)
text = ''.join(map(str, text)) 

wordcloud = WordCloud(width=1500, height=700, max_font_size=300,background_color='lightgrey').generate(text)
plt.figure(figsize=(20,17))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
fig = px.histogram(df, y="dayofweek",color="dayofweek")
fig.update_layout(
    title_text='Crime count on each day', 
    xaxis_title_text='Day',
    yaxis_title_text='Crimes Count', 
    bargap=0.5, 
    bargroupgap=0.5
)
fig.show()

<font color = "#B32636">
Let's look at the days of the week. The highest crime rate is seen on Friday. It seems that the crime was triggered by the fact that many people were out on Friday night due to Saturday being a holiday. Other days there is no significant difference, but Sunday has the lowest rate. 
Also, the fact that most of the people spend their Sunday at home may be a factor that reduces crime.

In [ ]:
sns.heatmap(pd.pivot_table(data = df, index = "dayofweek", 
                              columns = "Hour", values = "INCIDENT_NUMBER", aggfunc = 'count'), 
                              cmap = 'Reds')


<font color = "#B32636">
This time let's look at the time variable. The crime rate increases at noon (12:00) and after work hours (16: 00,17: 00,18: 00) on weekdays.
The crime rate increases at 00:00 on weekends, ie Fridays, Saturdays and Sundays.


In [ ]:
hist(df,'District','dayofweek','Day and District Distribution','Day','Districts')

<font color = "#B32636">
With this graph, it can be seen on which days and in which region more intense crimes were committed.
B2, C11 and D4 crime rates are at most 3 regions.

In [ ]:
df = df.dropna(axis=0)
print(df.isnull().sum(),"\nShape:",df.shape)

In [ ]:
hist(df,'District','UCR_PART','UCR and District Distribution','Day','UCR')

<font color = "#B32636">
Crimes are divided into 4 parts. Although the first two parties show different intensity in different districts, part 3 is the most committed crimes everywhere.
I focus on 2017 because the data is complete. The Motor Vehicle Accident Response with the highest crime rate has been visualized.The map below shows the district where this crime is most intensely committed.(B2)


In [ ]:
map_hooray = folium.Map(location=[42.361145,-71.057083],
                    zoom_start = 12, min_zoom=12) 

heat_df = df[df['Year']==2017]
heat_df =df[df['Group']=='Motor Vehicle Accident Response']
heat_df = df[['Lat','Long']]


heat_data = [[row['Lat'],row['Long']] for index, row in df.iterrows()]
HeatMap(heat_data, radius=10).add_to(map_hooray)
map_hooray

In [ ]:
B2_district=df.loc[df.District=='B2'][['Lat','Long']]
B2_district.Lat.fillna(0, inplace = True)
B2_district.Long.fillna(0, inplace = True) 

map_1=folium.Map(location=[42.356145,-71.064083], 
                 tiles = "OpenStreetMap",
                zoom_start=11)

folium.CircleMarker([42.319945,-71.079989],
                        radius=70,
                        fill_color="#b22222",
                        popup='Motor Vehicle Accident Response',
                        color='red',
                       ).add_to(map_1)


HeatMap(data=B2_district, radius=16).add_to(map_1)

map_1

<font color = "#B32636">
Now,let's see the areas where Larceny crime was found in 2016.
Then look at the district where this event is most common.(D4)

In [ ]:
map_hooray = folium.Map(location=[42.361145,-71.057083],
                    zoom_start = 12, min_zoom=12) 

heat_df = df[df['Year']==2016]
heat_df =df[df['Group']=='Larceny']
heat_df = df[['Lat','Long']]


heat_data = [[row['Lat'],row['Long']] for index, row in df.iterrows()]
HeatMap(heat_data, radius=10).add_to(map_hooray)
map_hooray

In [ ]:
B2_district=df.loc[df.District=='D4'][['Lat','Long']]
B2_district.Lat.fillna(0, inplace = True)
B2_district.Long.fillna(0, inplace = True) 

map_1=folium.Map(location=[42.356145,-71.064083], 
                 tiles = "OpenStreetMap",
                zoom_start=11)

folium.CircleMarker([42.319945,-71.079989],
                        radius=70,
                        fill_color="#b22222",
                        popup='Larceny',
                        color='red',
                       ).add_to(map_1)


HeatMap(data=B2_district, radius=16).add_to(map_1)

map_1

<font color = "#B32636">
Finally, I wanted to see other important crimes. This part is the crimes that I think are included in the UCR part two.

In [ ]:
rcParams["figure.figsize"] = 18,7
sns.set(font_scale=1.50)
order = df['Group'].value_counts().iloc[5:10].index
sns.countplot(data = df, x='Group',hue='District', order = order);


<p><img style="float: left;margin:50px 50px 50px 50px; max-width:400px" src="https://www.bhibcouncils.co.uk/wp-content/uploads/2020/04/vandalism.png"></p>


<p><img style="float: right;margin:50px 50px 50px 50px; max-width:400px" src="https://s3.amazonaws.com/law-media/uploads/922/60462/large/vusca.jpg?1540850613"></p>


<a id='libraries'></a>
<h1 style="color:firebrick" >4. General Analysis</h1>

* The lowest crime rate was in 2015 and the highest crime rate was in 2017.
* There is a serious decrease in crime rate in 2018. This may be due to the missing data of the last 3 months. However, if we consider the remaining 9 months, this is good news for humanity!

Well let's come to our question,Could we predict future crimes with past data?

* Yes.Each year, we encountered separate crimes in different districts, but there were also things that remained the same. For example, in all years, Friday, Saturday night and weekday lunch and departure times are the most prone to crime.

* Again, the crimes committed in the summer were much more than those committed in the winter months. We have seen that the seasons have an effect on crimes.

* We have seen that the districts with the highest number of crimes are B2, C11 and D4. In other words, as you can see from the maps, the most crime is committed in the city center.

* Especially, the most crimes have been done by motor vehicles. It can be said that the traffic rule has been violated. Sanctions can be reviewed again.(Note: Remember that most people who do not stop at the red light and drink driving etc. are not recorded in this data. There are actually more violators than here.)

* Another crime is Larceny. For this, the security forces can be informed by acting cautiously for the month, day and hour when the crime analyzed in the data is the most.

* Here, specific evaluations can also be made for districts and crimes that I have not analyzed, and action decisions can be taken accordingly.

* So yes, predictions can be made for the future based on past data.


<a id='libraries'></a>
<h1 style="color:firebrick" >5. Modelling</h1>

In [ ]:
crime=df.drop(['INCIDENT_NUMBER','Description', 'UCR_PART' , "Location",
                    'Date','quarter','dayofyear','dayofmonth','weekofyear',
                    'REPORTING_AREA','Street','dayofweek'] , axis=1)


In [ ]:
crime.head()

In [ ]:
def check_df(dataframe):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(3))
    print("##################### Tail #####################")
    print(dataframe.tail(3))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(crime)

In [ ]:
crime['Group'].value_counts().head(13)

In [ ]:
list_group = ('Motor Vehicle Accident Response',
              'Larceny',
              'Medical Assistance',
              'Investigate Person',
              'Other',
              'Drug Violation',
              'Simple Assault',
              'Vandalism',
              'Verbal Disputes',
              'Towed',
              'Investigate Property',
              'Larceny From Motor Vehicle',
              'Property Lost')

In [ ]:
crime_model = pd.DataFrame()

In [ ]:
i = 0

while i < len(list_group):

    crime_model  = crime_model.append(crime.loc[crime['Group'] == list_group[i]])
    
    i+=1

In [ ]:
list_column = ['District','Year','Month','Day','Hour','Lat','Long','Group']
crime_model = crime_model[list_column]

In [ ]:
crime_model['District'] = crime_model['District'].map({
    'B3':1, 
    'E18':2, 
    'B2':3, 
    'E5':4, 
    'C6':5, 
    'D14':6, 
    'E13':7, 
    'C11':8, 
    'D4':9, 
    'A7':10, 
    'A1':11, 
    'A15':12
})

crime_model['District'].unique()

In [ ]:
crime_model['Day'] = crime_model['Day'].map({
       'Tuesday':2, 
       'Saturday':6, 
       'Monday':1, 
       'Sunday':7, 
       'Thursday':4, 
       'Wednesday':3,
       'Friday':5
})

crime_model['Day'].unique()

In [ ]:
y=crime_model["Group"]
X=crime_model.drop("Group", axis=1)

In [ ]:
y = y.map({
    'Motor Vehicle Accident Response':1, 
    'Larceny':2, 
    'Medical Assistance':3,
    'Investigate Person':4, 
    'Other':5, 
    'Drug Violation':6, 
    'Simple Assault':7,
    'Vandalism':8, 
    'Verbal Disputes':9, 
    'Towed':10, 
    'Investigate Property':11,
    'Larceny From Motor Vehicle':12,
    'Property Lost':13
})

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X,
    y, 
    test_size = 0.1,
    random_state=42
)

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import *

lgbm= LGBMClassifier().fit(X_train,Y_train)
y_pred= lgbm.predict(X_test)

f1_score(Y_test, y_pred, average=None)

<font color = "#B32636">

## Note: Data is not prepared for modeling. The focus is on EDA and visualization.


<center><img
src="https://media1.giphy.com/media/xT5LMCrtGU8IhUenQs/source.gif" style="width:90%;height:30%;">
</center>
